## **LLM의 한계를 너머, Tool & Agent**

### **Tavily AI로 검색 기반 에이전트 구축하기**

**Tavily AI를 인터넷 검색 Tool로 설정**

In [ ]:
# !pip install langchain langchain_community langchain-openai pypdf langchain-text-splitters faiss-cpu

In [ ]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults

#Tavily AI API Key 환경변수 설정
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
os.environ['TAVILY_API_KEY']="YOUR_TAVILY_API_KEY"
tools = [TavilySearchResults()]

**Agent 구축**

In [ ]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent

#Agent에서 Tool을 활용하기 위한 프롬프트
prompt = hub.pull("hwchase17/openai-tools-agent")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
#LLM, Tool, 프롬프트를 사용하여 에이전트 생성
agent = create_openai_tools_agent(llm, tools, prompt)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "GPT-4o가 뭐야?"})

### **Langchain으로 벡터 DB & 인터넷 검색 Agent 구축하기**

**Tavily AI를 인터넷 검색 Tool로 설정**

In [ ]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults

#Tavily AI API Key 환경변수 설정
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
os.environ['TAVILY_API_KEY']="YOUR_TAVILY_API_KEY"
search = TavilySearchResults()

**벡터 DB 구축**

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("../data/★육아휴직제도 사용안내서_배포.pdf")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()


**create_retriever_tool로 벡터 DB 검색 툴 설정**

In [ ]:
from langchain.tools.retriever import create_retriever_tool
#벡터 DB 검색을 위한 검색 도구 설정
retriever_tool = create_retriever_tool(
    retriever,
    "parental_leave",
    "육아휴직과 관련한 정보를 검색합니다. 육아 휴직 관련한 질문이 입력되면 이 도구를 사용합니다.",
)

**Agent 구축**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor

tools = [search, retriever_tool]
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "육아휴직은 어떻게 사용할 수 있어?"})

**벡터DB를 활용하는 질문**

In [ ]:
agent_executor.invoke({"input": "육아휴직은 어떻게 사용할 수 있어?"})

**인터넷 검색을 활용하는 질문**

In [ ]:
agent_executor.invoke({"input": "현재 대한민국의 인구는 몇명이야?"})